# Data Acquisition
This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.

- Read the case, department, and source data into their own spark dataframes.

- Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?
- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [1]:
import pyspark
from pyspark.sql.functions import *

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
case = spark.read.csv('case.csv', header=True, inferSchema=True)
dept = spark.read.csv('dept.csv', header=True, inferSchema=True)
source = spark.read.csv('source.csv', header=True, inferSchema=True)

In [4]:
case.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [5]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

# Data Preparation

## Rename the column name

In [6]:
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

## Change columns to a bool type

In [7]:
case = (
    case
    .withColumn('case_late', col('case_late')== 'YES')
    .withColumn('case_closed', col('case_closed')== 'YES')
)

## Change date to timestamp

In [8]:
case =(
    case
    .withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

In [9]:
case.show(1, vertical =True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



##  Join other tables 

In [10]:
source.show(1, vertical= True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [11]:
dept.show(1, vertical = True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [12]:
df = (
    case
    .join(dept, 'dept_division', 'left')
    .join(source, 'source_id', 'left')
)

In [13]:
df.show(1, vertical =True)

-RECORD 0--------------------------------------
 source_id              | svcCRMLS             
 dept_division          | Field Operations     
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 case_due_date          | 2020-09-26 00:42:00  
 case_late              | false                
 num_days_late          | -998.5087616000001   
 case_closed            | true                 
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMLS             
only showing top 1 row



# Data Exploration Exercise

- How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [14]:
(
    df
    .where(col("num_days_late").isNotNull())
    .filter(col('case_closed')=='false')
    .sort(col("case_opened_date").desc())
    .show(1, vertical=True)
)

-RECORD 0--------------------------------------
 source_id              | CRM_Listener         
 dept_division          | Code Enforcement     
 case_id                | 1014759619           
 case_opened_date       | 2018-08-08 10:38:00  
 case_closed_date       | null                 
 case_due_date          | 2018-08-22 10:38:00  
 case_late              | false                
 num_days_late          | -13.44305556         
 case_closed            | false                
 service_request_type   | Bandit Signs         
 SLA_days               | 14.0                 
 case_status            | Open                 
 request_address        | 1935  MILITARY DR... 
 council_district       | 3                    
 dept_name              | Code Enforcement ... 
 standardized_dept_name | DSD/Code Enforcement 
 dept_subject_to_SLA    | YES                  
 source_username        | CRM_Listener         
only showing top 1 row



- How many Stray Animal cases are there?

In [15]:
df\
.filter(col('service_request_type')=='Stray Animal')\
.count()\


27361

- How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [28]:
df\
.filter(col('dept_division')=='Field Operations')\
.filter(col('service_request_type')!='Officer Standby')\
.count()

116295

In [17]:
unique_service_type = df\
.filter(col('dept_division')=='Field Operations')\
.filter(col('service_request_type')!='Officer Standby')\
.select('service_request_type')\
.distinct()

unique_service_type.show(truncate=False)
unique_service_type.count()

+-------------------------------+
|service_request_type           |
+-------------------------------+
|Animal Permits Request         |
|Injured Animal(Critical)       |
|Animal Bite(Non-Critical)      |
|Stray Animal                   |
|Trapped/Confined Animal        |
|City Council Animal Request    |
|Aggressive Animal(Non-Critical)|
|Animal Bite(Critical)          |
|Public Nuisance(Own Animal)    |
|Aggressive Animal(Critical)    |
|Animal Cruelty(Critical)       |
|Spay/Neuter Request Response   |
|Animal Neglect                 |
+-------------------------------+



13

- Convert the council_district column to a string column.

In [18]:
df = df.withColumn('council_district', col('council_district').cast('string'))
df.select('council_district').dtypes

[('council_district', 'string')]

- Extract the year from the case_closed_date column.

In [19]:
df = df.withColumn('year', year('case_closed_date'))
df.show(1,vertical=True)

-RECORD 0--------------------------------------
 source_id              | svcCRMLS             
 dept_division          | Field Operations     
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 case_due_date          | 2020-09-26 00:42:00  
 case_late              | false                
 num_days_late          | -998.5087616000001   
 case_closed            | true                 
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMLS             
 year                   | 2018                 
only showing top 1 row



- Convert num_days_late from days to hours in new columns num_hours_late.

In [20]:
df = df.withColumn('num_hours_late', col('num_days_late')*24)
df.show(1,vertical=True)

-RECORD 0--------------------------------------
 source_id              | svcCRMLS             
 dept_division          | Field Operations     
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 case_due_date          | 2020-09-26 00:42:00  
 case_late              | false                
 num_days_late          | -998.5087616000001   
 case_closed            | true                 
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMLS             
 year                   | 2018                 
 num_hours_late         | -23964.2102784

- Are there any cases that do not have a request source?

In [21]:
df.where(col('source_id').isNull()).count()

0

- What are the top 10 service request types in terms of number of requests?

In [22]:
(
    df
    .groupBy('service_request_type')
    .count()
    .sort(col("count").desc())
    .show(10)
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



- What are the top 10 service request types in terms of average days late?

In [23]:
(
    df
    .where(col("num_days_late").isNotNull())
    .groupBy('service_request_type')
    .agg(avg('num_days_late').alias('average_days_late'))
    .sort(col('average_days_late').desc())
    .show(10, truncate =False)
)

+--------------------------------------+------------------+
|service_request_type                  |average_days_late |
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.95636210420943|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.11255641500003|
|Zoning: Recycle Yard                  |135.92851612479797|
|Donation Container Enforcement        |131.75610506358706|
|License Requied Used Mattress Sales   |128.79828704142858|
|Traffic Signal Graffiti               |101.79846062200002|
|Complaint                             |72.8705023031169  |
+--------------------------------------+------------------+
only showing top 10 rows



- Does number of days late depend on department?

In [29]:
(
    df
    .filter('case_late') # just look at the late cases.
    .where(col("num_days_late").isNotNull())
    .groupBy('standardized_dept_name')
    .agg(avg('num_days_late').alias('average_days_late'))
    .sort(col('average_days_late').desc())
    .show(10, truncate =False)
)

+------------------------+------------------+
|standardized_dept_name  |average_days_late |
+------------------------+------------------+
|Customer Service        |88.18248182589822 |
|DSD/Code Enforcement    |49.506339986350454|
|Animal Care Services    |23.4467296347382  |
|Parks & Recreation      |22.427807192724135|
|Trans & Cap Improvements|10.662950455078674|
|Solid Waste             |7.1471727895574135|
|Metro Health            |6.494699602827871 |
+------------------------+------------------+



- How do number of days late depend on department and request type?

In [25]:
(
    df
    .where(col("num_days_late").isNotNull())
    .groupBy('standardized_dept_name','service_request_type')
    .agg(avg('num_days_late').alias('average_days_late'))
    .sort(col('average_days_late').desc())
    .show(10)
)

+----------------------+--------------------+------------------+
|standardized_dept_name|service_request_type| average_days_late|
+----------------------+--------------------+------------------+
|  DSD/Code Enforcement|  Zoning: Junk Yards|175.95636210420943|
|  DSD/Code Enforcement|Labeling for Used...|162.43032902285717|
|  DSD/Code Enforcement|Record Keeping of...|153.99724039428568|
|  DSD/Code Enforcement|Signage Requied f...|151.63868055333333|
|  DSD/Code Enforcement|Storage of Used M...|142.11255641500003|
|  DSD/Code Enforcement|Zoning: Recycle Yard|135.92851612479797|
|  DSD/Code Enforcement|Donation Containe...|131.75610506358706|
|  DSD/Code Enforcement|License Requied U...|128.79828704142858|
|  Trans & Cap Impro...|Traffic Signal Gr...|101.79846062200002|
|      Customer Service|           Complaint|  72.8705023031169|
+----------------------+--------------------+------------------+
only showing top 10 rows



- You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [26]:
df.show(1,vertical =True)

-RECORD 0--------------------------------------
 source_id              | svcCRMLS             
 dept_division          | Field Operations     
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 case_due_date          | 2020-09-26 00:42:00  
 case_late              | false                
 num_days_late          | -998.5087616000001   
 case_closed            | true                 
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMLS             
 year                   | 2018                 
 num_hours_late         | -23964.2102784

In [27]:
max_date = df.agg(max('case_closed_date'))
max_date.show()

+---------------------+
|max(case_closed_date)|
+---------------------+
|  2018-08-08 10:38:00|
+---------------------+

